### RB SmartCard

In [18]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import sys
from os.path import join
from bs4 import BeautifulSoup
from importlib import reload
import codecs
import re

import roamability as rb

downloads = 'C:/Users/balob/Downloads'

rb.greetings_func()

Greetings from Roamability!!! 



In [8]:
reload(rb)

<module 'roamability' from 'C:\\Users\\balob\\Documents\\GITLAB\\RB_BD\\roamability.py'>

In [ ]:
class SmartCard:
    
    def __init__(self):
        pass

In [10]:
# Send SS7 message

ss7 = rb.SendMsgSs7()
# ss7.sri4sm(ss7.ogt['partner'], '79217428080', '79217428080')
# ss7.print_imsi_prof(ss7.ogt['partner'], '425019613000536', '79219600320', 'as_resp')
# ss7.sendSMS(ss7.ogt['partner'], '425019613000536', '79219600320', ss7.get_config)

In [16]:
# IMSI decoder

imsi = '4952106941005106'
imsi = '425019614001560'

def imsi_conv(imsi, code='code'):
    """Conver deconvert IMSI for perso. code='code' / 'decode'"""
    if code=='code':
        imsi = '9' + imsi
        return ''.join([imsi[i] for i in [1, 0, 3, 2, 5, 4, 7, 6, 9, 8, 11, 10, 13, 12, 15, 14]])
    elif code=='decode':
        return ''.join([imsi[i] for i in [0, 3, 2, 5, 4, 7, 6, 9, 8, 11, 10, 13, 12, 15, 14]])

# imsi_conv(imsi, code='decode')
imsi_conv(imsi, code='code')

'4952106941005106'

In [17]:
# ANY decoder

value = '4952106941005106'

def any_decoder(value):
    value_conv = ''
    for i in range(1, len(value), 2):
        value_conv += value[i: i + 1]
        value_conv += value[i - 1: i]
    return value_conv

any_decoder(value)

'9425019614001560'

In [22]:
# ASCII decoder

value = b"Smartel"
value = '524F414D4142494C49' # 'ROAMABILI'

def ascii_conv(value, code='code'):
    if code=='code':
        return codecs.encode(value, "hex").upper()
    elif code=='decode':
        return bytearray.fromhex(value).decode()

# ascii_conv(value, code='code')
ascii_conv(value, code='decode')

'ROAMABILI'

In [ ]:
l = len('42F004C08042F020C00042F070C00042F010C00042F042C0C042F030C08042F070008042F01000C0FFFFFF0000FFFFFF0000') / 2
print('Dec to hex')
print(f'{int(l)} -> 0x{int(l):X}')

In [ ]:
ota_env =\
"""
D1 51 82 02  83 81 06 04   01 99 99 99  8B 45 40 06   
01 88 88 88  7F F6 02 70   20 22 23 11  30 35 02 70   
00 00 30 15  16 01 19 19   B0 00 11 55  2C 28 B7 60   
04 80 D0 33  56 08 78 29   5A F4 7B 5C  12 A1 7A 00   
73 9B 60 A3  CB A0 58 4F   B5 03 61 B9  6C 38 B7 47   
F0 13 17 
"""

def ota_cmd(ota_env):
    ota_env = ota_env.replace(' ', '').replace('\n', '')
    print(f'{ota_env}')
    print(f'send_apdu -APDU A0C20000{int(len(ota_env)/2):X}{ota_env}')

ota_cmd(ota_env)

In [ ]:
# GET CONFIG decoder

config = '010301001E001000000003000305010202000000'
config = '010301001E001000000003000305010202000000'

# config = '010301001E011400000003000305010200000000' # default profile order  01 00
# config = '010300001E011400000003000305010201000000' # order 00 01

applet_enabling_flag = config[0:2]
applet_enabling_flag_dict = {'01':'Activate','00':'Deactivate'}

mode = config[2:4]
mode_dict = {'01':'Events only mode','02':'Timer only mode','03':'Events & Timer mode'}

default_imsi = config[4:6]
default_imsi_dict = {'00':'IMSI0','01':'IMSI1','02':'IMSI2','03':'IMSI3','04':'IMSI4','05':'IMSI5'}

polling_period = config[6:10]

features_activation_flags = config[10:14]

scan_delay_timer = config[22:26]

print(f'Applet enabling flag: {applet_enabling_flag_dict[applet_enabling_flag]}')
print(f'Mode: {mode_dict[mode]}')
print(f'Default IMSI: {default_imsi_dict[default_imsi]}')
print(f'Polling period: {int(polling_period, 16)} sec.')
print(f'Features activation flags: {int(features_activation_flags, 16):0>16b}')
# Features activation flags 1(1 Enable, 0 Disable):
# b1:Start with Primary (После рестарта UE будет выбран профайл 0).
# b2:Reset Loci, PS Loci (обнуление TIMSI).
# b3:RB_MODE (В HOME MCC будут перебираться IMSI спонсора).
# b4 Skip Primary IMSI in roaming: (Исключение Home IMSI в роуминге).
# b5:
# b6-b16: RFU
print(f'Scan delay timer: {int(scan_delay_timer, 16)} sec.')

### Developments

In [7]:
class TestClass:
    c_var01 = 'TestClassVar'
    def __init__(self, i_var):
        self.i_var01 = i_var
        self.i_var02 = self.c_var01

test_class = TestClass('TestInstanceVar')

In [10]:
TestClass.c_var01 = 'TestClassVar01'
test_class.i_var02
test_class.i_var01

'TestInstanceVar'